In [200]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

from datetime import date
import glob

path = r'data/output/'  # use your path
allFiles = glob.glob(path + "/2*.csv")

dftrain = pd.DataFrame()

for _file in allFiles:
        try:
            dftrain = pd.concat([dftrain,pd.read_csv(_file,header=0)])
        except:
            pass
#dftrain = pd.read_csv('data/extras/train.csv', header=0)
dftest = pd.read_csv('data/output/3017-01-21.csv', header=0)

In [201]:
df_train = dftrain

scaler = StandardScaler()
features = ['Salary','AvgPointsPerGame','atHome','O/U','odds']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train["Final"].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [202]:
df_test = dftest

X_test = scaler.transform(df_test[features].values)
y_test = df_test["Final"].values

In [208]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=92,
                              random_state=1,
                              verbose=False,
                              warm_start=True)
modelRFR = modelRFR.fit(X_train, y_train)

y_tree_prediction = modelRFR.predict(X_test)
print ("\naccuracy", np.sum(abs(y_tree_prediction-y_test)<5) / float(len(y_test)))


accuracy 0.425170068027


In [162]:
#pd.concat([pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [211]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(12, input_dim=5, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
          
model.compile(optimizer='adam', metrics=['accuracy'],loss="mean_squared_error")

model.fit(X_train, y_train, 
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=150, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 75.08956599235535 seconds


In [212]:
scores = model.evaluate(X_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/294 [==>...........................] - ETA: 0s

acc: 2.04%


In [218]:
dfpredict = pd.read_csv('data/output/3017-01-23.csv', header=0)

X_test = scaler.transform(dfpredict[features].values)

y_predict = model.predict(X_test)
y_tree_predict = modelRFR.predict(X_test)

In [235]:
output = pd.concat([dfpredict,
            pd.DataFrame(y_predict,columns=["NN2"]),
             pd.DataFrame(y_tree_predict,columns=["RF2"])]
          ,axis=1)
output.to_csv('data/output/2017-01-23.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [237]:
output.loc[:,("Name","NN2","RF2","AvgPointsPerGame")]

,Name,NN2,RF2,AvgPointsPerGame
0,James Harden,58.393467,62.203804,60.617
1,Russell Westbrook,55.097851,58.456522,64.080
2,Anthony Davis,53.256474,48.836957,54.439
3,Giannis Antetokounmpo,52.113594,53.777174,50.089
4,DeMarcus Cousins,49.345375,48.543478,52.863
5,Kevin Durant,47.883415,51.989130,50.244
6,LeBron James,47.238647,47.043478,51.346
7,Kawhi Leonard,44.359371,42.956522,42.732
8,John Wall,43.522820,46.293478,48.896
9,Stephen Curry,41.148499,45.703804,43.352
